In [1]:
import pandas as pd
import numpy as np
from sklearn.utils import resample
import math

In [2]:
d_labitems = pd.read_csv('data/mimic-iv-0.4/hosp/d_labitems.csv.gz', compression='gzip')

In [3]:
admissions_sample = pd.read_pickle("admissions_sample.pkl")

## Pull Chart and Lab data only where it matches our sample patient list

In [4]:
# # We have Decided not to use Chart data after exploring it further
# # However, it still comes up in our discussions, so keeping this for now
# # Will need to update this code based on other updates we've made

# %%time #run this for next time

# chunksize = 3*(10 ** 6)
# counter=0
# chartevents_sample = []
# for chunk in pd.read_csv('data/mimic-iv-0.4/icu/chartevents.csv.gz', compression='gzip', chunksize=chunksize):
#     chartevents_sample.append(chunk[chunk['subject_id'].isin(list(admissions_sample.subject_id))])
#     counter+=1
#     print(f'chunk {counter} processed')

# chartevents_final = pd.concat(chartevents_sample)



In [5]:
# chartevents_final.to_pickle("chartevents_final.pkl")

In [6]:
# chartevents_final = pd.read_pickle("chartevents_final.pkl")

In [7]:
# chartevents_final.shape

Load Lab Events table

In [8]:
# %%time

# chunksize = 3*(10 ** 6)
# counter=0
# lab_events_sample = []
# for chunk in pd.read_csv('data/mimic-iv-0.4/hosp/labevents.csv.gz', compression='gzip', chunksize=chunksize):
#     lab_events_sample.append(chunk[chunk['subject_id'].isin(list(admissions_sample.subject_id))])
#     counter+=1
#     print(f'chunk {counter} processed')

# lab_events_final = pd.concat(lab_events_sample)
# # 5k patients
# # 3min 52s

# # 10k patients
# # 4min 6s



In [9]:
# lab_events_final.to_pickle("lab_events_final.pkl")

In [10]:
lab_events_final = pd.read_pickle("lab_events_final.pkl")

In [11]:
lab_events_final.shape

(4573368, 15)

## Manipulating the Data

Take only records related to our sample patients

Filter lab events based on first hospital visit of our sample population

In [12]:
# Can we do this filtering when generating lab data?
# For now let's not as we're discussing using different hospitalizations from the same subjects

%time lab_events_sampled = lab_events_final[lab_events_final['hadm_id'].isin(admissions_sample.hadm_id)]


CPU times: user 1.14 s, sys: 490 ms, total: 1.63 s
Wall time: 1.64 s


Check how many patients have had each test done

In [13]:
lab_events_sampled.groupby('itemid')['subject_id'].nunique().sort_values(ascending = False).head(10)

itemid
51221    6187
51265    6093
51222    6040
51301    6038
51248    6034
51249    6034
51277    6034
51279    6034
51250    6034
50971    5548
Name: subject_id, dtype: int64

Most common tests performed on patients with pos diagnosis

In [14]:
lab_events_sampled[lab_events_sampled['subject_id'].isin(list(admissions_sample[admissions_sample['pos'] == 1].subject_id))].groupby('itemid')['subject_id'].nunique().sort_values(ascending = False).head(10)


itemid
50971    799
50912    797
50983    797
51006    796
50902    796
50868    792
50882    792
51265    789
51221    789
50931    788
Name: subject_id, dtype: int64

Item Black List

In [15]:
item_black_list = [50920]
# 50920 - test results are text in comments - need to come back and figure out how to handle this


Take only the X most commonly performed tests

In [16]:
top_test_num = 40

lab_events_valid = lab_events_sampled[~lab_events_sampled['itemid'].isin(item_black_list)]

# On patients with pos diagnosis
itemid_sub_sample = lab_events_valid[lab_events_valid['subject_id'].isin(list(admissions_sample[admissions_sample['pos'] == 1].subject_id))].groupby('itemid')['subject_id'].nunique().sort_values(ascending = False).head(top_test_num).reset_index().rename(columns = {'index' : 'itemid'}).itemid.to_list()
# On all sample patients
# itemid_sub_sample = lab_events_sampled.groupby('itemid')['subject_id'].nunique().sort_values(ascending = False).head(top_test_num).reset_index().rename(columns = {'index' : 'itemid'}).itemid.to_list()

lab_events_sampled_sub = lab_events_sampled[lab_events_sampled['itemid'].isin(itemid_sub_sample)]


## Data Cleaning

Make a separate table where we run some of the forest models on the Text data?

Special updates for Urine sample tests

In [17]:
# Other invalid value imputation
# For now set them all to normal/most common value
lab_events_sampled_sub.loc[(lab_events_sampled_sub['itemid'] == 51237) & (lab_events_sampled_sub['value'].isin([np.nan, 'ERROR', 'UNABLE TO REPORT'])), ['value', 'valuenum']] = 1.1
lab_events_sampled_sub.loc[(lab_events_sampled_sub['itemid'] == 51237) & (lab_events_sampled_sub['value'].isin(['>21.8'])), ['value', 'valuenum']] = 22
lab_events_sampled_sub.loc[(lab_events_sampled_sub['itemid'] == 51237) & (lab_events_sampled_sub['value'].isin(['>19.2'])), ['value', 'valuenum']] = 20

lab_events_sampled_sub.loc[(lab_events_sampled_sub['itemid'] == 51275) &  (lab_events_sampled_sub['value'].isin(['>150', '>150.0'])), ['value', 'valuenum']] = 175

# Should we just delete these?
# For 5k patients there are 6 records here
lab_events_sampled_sub.loc[(lab_events_sampled_sub['itemid'] == 51274) &  (lab_events_sampled_sub['value'].isin(['ERROR'])), ['value', 'valuenum']] = 11

# Should we just delete these?
# For 5k patients there are 1 records here
lab_events_sampled_sub.loc[(lab_events_sampled_sub['itemid'] == 51275) &  (lab_events_sampled_sub['value'].isin(['UNABLE TO REPORT'])), ['value', 'valuenum']] = 31

# Clumsy..
lab_events_sampled_sub.loc[(lab_events_sampled_sub['itemid'] == 51275) &  (lab_events_sampled_sub['value'].isin(['34..3'])), ['value', 'valuenum']] = 34.3


/opt/anaconda3/lib/python3.7/site-packages/pandas/core/indexing.py:965: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  self.obj[item] = s


More Data Cleaning

In [18]:
%%time

# Can probably make a helper function combining some of the below mapping

# 51466
lab_events_sampled_sub.loc[(lab_events_sampled_sub['itemid'] == 51466) & (lab_events_sampled_sub['value'].isin([np.nan])), ['value']] = lab_events_sampled_sub['comments']
lab_events_sampled_sub.loc[(lab_events_sampled_sub['itemid'] == 51466) & (lab_events_sampled_sub['value'].isin(['NEG.  ', 'NEG', ' '])), ['value', 'valuenum']] = 0
lab_events_sampled_sub.loc[(lab_events_sampled_sub['itemid'] == 51466) & (lab_events_sampled_sub['value'].isin(['TR', 'TR.  ', 'TR*.  '])), ['value', 'valuenum']] = 1
lab_events_sampled_sub.loc[(lab_events_sampled_sub['itemid'] == 51466) & (lab_events_sampled_sub['value'].isin(['SM', 'SM .  ', 'SM*.  '])), ['value', 'valuenum']] = 2
lab_events_sampled_sub.loc[(lab_events_sampled_sub['itemid'] == 51466) & (lab_events_sampled_sub['value'].isin(['MOD', 'MOD.  ', 'MOD*.  '])), ['value', 'valuenum']] = 3
lab_events_sampled_sub.loc[(lab_events_sampled_sub['itemid'] == 51466) & (lab_events_sampled_sub['value'].isin(['LG', 'LG.  ', 'LG*.  ', 'LGE', 'LRG'])), ['value', 'valuenum']] = 4

lab_events_sampled_sub.loc[(lab_events_sampled_sub['itemid'] == 51466) & (lab_events_sampled_sub['ref_range_lower'].isnull()), 'ref_range_lower'] = 0
lab_events_sampled_sub.loc[(lab_events_sampled_sub['itemid'] == 51466) & (lab_events_sampled_sub['ref_range_upper'].isnull()), 'ref_range_upper'] = 0

# 51514
lab_events_sampled_sub.loc[(lab_events_sampled_sub['itemid'] == 51514) & (lab_events_sampled_sub['value'].isin([np.nan])), ['value']] = lab_events_sampled_sub['comments']
lab_events_sampled_sub.loc[(lab_events_sampled_sub['itemid'] == 51514) & (lab_events_sampled_sub['value'].isin(['NEG.  ', 'NEG', 'NORMAL.  ', ' '])), ['value', 'valuenum']] = 0
lab_events_sampled_sub.loc[(lab_events_sampled_sub['itemid'] == 51514) & (lab_events_sampled_sub['value'].isin(['>8'])), ['value', 'valuenum']] = 10
lab_events_sampled_sub.loc[(lab_events_sampled_sub['itemid'] == 51514) & (lab_events_sampled_sub['value'].isin(['>12.  ', '>12*.  '])), ['value', 'valuenum']] = 15

# 51464
lab_events_sampled_sub.loc[(lab_events_sampled_sub['itemid'] == 51464) & (lab_events_sampled_sub['value'].isin([np.nan])), ['value']] = lab_events_sampled_sub['comments']
lab_events_sampled_sub.loc[(lab_events_sampled_sub['itemid'] == 51464) & (lab_events_sampled_sub['value'].isin(['NEG.  ', 'NEG', ' '])), ['value', 'valuenum']] = 0
lab_events_sampled_sub.loc[(lab_events_sampled_sub['itemid'] == 51464) & (lab_events_sampled_sub['value'].isin(['SM', 'SM .  ', 'SM*.  '])), ['value', 'valuenum']] = 1
lab_events_sampled_sub.loc[(lab_events_sampled_sub['itemid'] == 51464) & (lab_events_sampled_sub['value'].isin(['MOD', 'MOD.  ', 'MOD*.  '])), ['value', 'valuenum']] = 2
lab_events_sampled_sub.loc[(lab_events_sampled_sub['itemid'] == 51464) & (lab_events_sampled_sub['value'].isin(['LG', 'LG.  ', 'LG*.  ', 'LGE'])), ['value', 'valuenum']] = 3

lab_events_sampled_sub.loc[(lab_events_sampled_sub['itemid'] == 51464) & (lab_events_sampled_sub['ref_range_lower'].isnull()), 'ref_range_lower'] = 0
lab_events_sampled_sub.loc[(lab_events_sampled_sub['itemid'] == 51464) & (lab_events_sampled_sub['ref_range_upper'].isnull()), 'ref_range_upper'] = 0

# 51487
# Should Pos be marked abnormal?
lab_events_sampled_sub.loc[(lab_events_sampled_sub['itemid'] == 51487) & (lab_events_sampled_sub['value'].isin([np.nan])), ['value']] = lab_events_sampled_sub['comments']
lab_events_sampled_sub.loc[(lab_events_sampled_sub['itemid'] == 51487) & (lab_events_sampled_sub['value'].isin(['NEG.  ', 'NEG', ' '])), ['value', 'valuenum']] = 0
lab_events_sampled_sub.loc[(lab_events_sampled_sub['itemid'] == 51487) & (lab_events_sampled_sub['value'].isin(['POS.  ', 'POS', 'POS*.  '])), ['value', 'valuenum']] = 1

lab_events_sampled_sub.loc[(lab_events_sampled_sub['itemid'] == 51487) & (lab_events_sampled_sub['ref_range_lower'].isnull()), 'ref_range_lower'] = 0
lab_events_sampled_sub.loc[(lab_events_sampled_sub['itemid'] == 51487) & (lab_events_sampled_sub['ref_range_upper'].isnull()), 'ref_range_upper'] = 0

# 51492
# Should Pos be marked abnormal?
lab_events_sampled_sub.loc[(lab_events_sampled_sub['itemid'] == 51492) & (lab_events_sampled_sub['value'].isin([np.nan])), ['value']] = lab_events_sampled_sub['comments']
lab_events_sampled_sub.loc[(lab_events_sampled_sub['itemid'] == 51492) & (lab_events_sampled_sub['value'].isin(['___'])), ['value', 'valuenum']] = np.nan
lab_events_sampled_sub.loc[(lab_events_sampled_sub['itemid'] == 51492) & (lab_events_sampled_sub['value'].isin(['NEG.  ', 'NEG', ' '])), ['value', 'valuenum']] = 0
lab_events_sampled_sub.loc[(lab_events_sampled_sub['itemid'] == 51492) & (lab_events_sampled_sub['value'].isin(['TR', 'TR.  ', 'TR*.  '])), ['value', 'valuenum']] = 10
lab_events_sampled_sub.loc[(lab_events_sampled_sub['itemid'] == 51492) & (lab_events_sampled_sub['value'].str.contains('TR.')), ['value', 'valuenum']] = 10
lab_events_sampled_sub.loc[(lab_events_sampled_sub['itemid'] == 51492) & (lab_events_sampled_sub['value'].isin(['>300'])), ['value', 'valuenum']] = 350
lab_events_sampled_sub.loc[(lab_events_sampled_sub['itemid'] == 51492) & (lab_events_sampled_sub['value'].isin(['>600.  ', '>600*.  '])), ['value', 'valuenum']] = 700

lab_events_sampled_sub.loc[(lab_events_sampled_sub['itemid'] == 51492) & (lab_events_sampled_sub['ref_range_lower'].isnull()), 'ref_range_lower'] = 0
lab_events_sampled_sub.loc[(lab_events_sampled_sub['itemid'] == 51492) & (lab_events_sampled_sub['ref_range_upper'].isnull()), 'ref_range_upper'] = 0

# 51486
lab_events_sampled_sub.loc[(lab_events_sampled_sub['itemid'] == 51486) & (lab_events_sampled_sub['value'].isin([np.nan])), ['value']] = lab_events_sampled_sub['comments']
lab_events_sampled_sub.loc[(lab_events_sampled_sub['itemid'] == 51486) & (lab_events_sampled_sub['value'].isin(['NEG.  ', 'NEG', ' '])), ['value', 'valuenum']] = 0
lab_events_sampled_sub.loc[(lab_events_sampled_sub['itemid'] == 51486) & (lab_events_sampled_sub['value'].isin(['TR', 'TR.  ', 'TR*.  '])), ['value', 'valuenum']] = 1
lab_events_sampled_sub.loc[(lab_events_sampled_sub['itemid'] == 51486) & (lab_events_sampled_sub['value'].isin(['SM', 'SM .  ', 'SM*.  '])), ['value', 'valuenum']] = 2
lab_events_sampled_sub.loc[(lab_events_sampled_sub['itemid'] == 51486) & (lab_events_sampled_sub['value'].isin(['MOD', 'MOD.  ', 'MOD*.  '])), ['value', 'valuenum']] = 3
lab_events_sampled_sub.loc[(lab_events_sampled_sub['itemid'] == 51486) & (lab_events_sampled_sub['value'].isin(['LG', 'LG.  ', 'LG*.  ', 'LGE'])), ['value', 'valuenum']] = 4

lab_events_sampled_sub.loc[(lab_events_sampled_sub['itemid'] == 51486) & (lab_events_sampled_sub['ref_range_lower'].isnull()), 'ref_range_lower'] = 0
lab_events_sampled_sub.loc[(lab_events_sampled_sub['itemid'] == 51486) & (lab_events_sampled_sub['ref_range_upper'].isnull()), 'ref_range_upper'] = 0

# 51506
lab_events_sampled_sub.loc[(lab_events_sampled_sub['itemid'] == 51506) & (lab_events_sampled_sub['value'].isin([np.nan])), ['value']] = lab_events_sampled_sub['comments']
lab_events_sampled_sub.loc[(lab_events_sampled_sub['itemid'] == 51506) & (lab_events_sampled_sub['value'].isin(['Clear.  ', 'CLEAR.  ', 'Clear', ' ', 'CLEAR'])), ['value', 'valuenum']] = 0
lab_events_sampled_sub.loc[(lab_events_sampled_sub['itemid'] == 51506) & (lab_events_sampled_sub['value'].isin(['SlHazy'])), ['value', 'valuenum']] = 0.5
lab_events_sampled_sub.loc[(lab_events_sampled_sub['itemid'] == 51506) & (lab_events_sampled_sub['value'].isin(['Hazy.  ', 'Hazy*.  ', 'Hazy', 'HAZY', 'HAZY*.  ', 'Slcldy'])), ['value', 'valuenum']] = 1
lab_events_sampled_sub.loc[(lab_events_sampled_sub['itemid'] == 51506) & (lab_events_sampled_sub['value'].isin(['Cloudy*.  ', 'Cloudy.  ', 'Cloudy', 'CLO', 'CLOUDY', 'CLOU', 'CLOUDY*.  '])), ['value', 'valuenum']] = 2

lab_events_sampled_sub.loc[(lab_events_sampled_sub['itemid'] == 51506) & (lab_events_sampled_sub['ref_range_lower'].isnull()), 'ref_range_lower'] = 0
lab_events_sampled_sub.loc[(lab_events_sampled_sub['itemid'] == 51506) & (lab_events_sampled_sub['ref_range_upper'].isnull()), 'ref_range_upper'] = 0


CPU times: user 2.63 s, sys: 63.2 ms, total: 2.7 s
Wall time: 2.71 s


Additional Data Cleaning

In [19]:
lab_events_sampled_sub.loc[(lab_events_sampled_sub['itemid'] == 51484) & (lab_events_sampled_sub['value'].isin([np.nan, ' ', 'TR', 'NEG'])), ['value', 'valuenum']] = 0
lab_events_sampled_sub.loc[(lab_events_sampled_sub['itemid'] == 51478) & (lab_events_sampled_sub['value'].isin([np.nan, ' ', 'TR', 'NEG'])), ['value', 'valuenum']] = 0

lab_events_sampled_sub.loc[(lab_events_sampled_sub['itemid'] == 51484) & (lab_events_sampled_sub['ref_range_lower'].isnull()), 'ref_range_lower'] = 0
lab_events_sampled_sub.loc[(lab_events_sampled_sub['itemid'] == 51478) & (lab_events_sampled_sub['ref_range_lower'].isnull()), 'ref_range_lower'] = 0

lab_events_sampled_sub.loc[(lab_events_sampled_sub['itemid'] == 51484) & (lab_events_sampled_sub['ref_range_upper'].isnull()), 'ref_range_upper'] = 0
lab_events_sampled_sub.loc[(lab_events_sampled_sub['itemid'] == 51478) & (lab_events_sampled_sub['ref_range_upper'].isnull()), 'ref_range_upper'] = 0

# What are good replacement values for these?
lab_events_sampled_sub.loc[(lab_events_sampled_sub['itemid'] == 51484) & (lab_events_sampled_sub['value'].isin(['>80'])), ['value', 'valuenum']] = 120
lab_events_sampled_sub.loc[(lab_events_sampled_sub['itemid'] == 51478) & (lab_events_sampled_sub['value'].isin(['>1000'])), ['value', 'valuenum']] = 1250

lab_events_sampled_sub.loc[(lab_events_sampled_sub['itemid'] == 51498) & (lab_events_sampled_sub['value'].isin(['<1.005'])), ['value', 'valuenum']] = 1.000
lab_events_sampled_sub.loc[(lab_events_sampled_sub['itemid'] == 51498) & (lab_events_sampled_sub['value'].isin(['<=1.005'])), ['value', 'valuenum']] = 1.000
lab_events_sampled_sub.loc[(lab_events_sampled_sub['itemid'] == 51498) & (lab_events_sampled_sub['value'].isin(['>1.030'])), ['value', 'valuenum']] = 1.035
lab_events_sampled_sub.loc[(lab_events_sampled_sub['itemid'] == 51498) & (lab_events_sampled_sub['value'].isin(['>1.035'])), ['value', 'valuenum']] = 1.040
lab_events_sampled_sub.loc[(lab_events_sampled_sub['itemid'] == 51498) & (lab_events_sampled_sub['value'].isin(['>1.050'])), ['value', 'valuenum']] = 1.055
lab_events_sampled_sub.loc[(lab_events_sampled_sub['itemid'] == 51498) & (lab_events_sampled_sub['value'].isin(['>=1.035'])), ['value', 'valuenum']] = 1.040
lab_events_sampled_sub.loc[(lab_events_sampled_sub['itemid'] == 51498) & (lab_events_sampled_sub['value'].isin([' '])), ['value', 'valuenum']] = 1.015

lab_events_sampled_sub.loc[(lab_events_sampled_sub['itemid'] == 51237) & (lab_events_sampled_sub['value'].isin(['>13.4'])), ['value', 'valuenum']] = 15

lab_events_sampled_sub.loc[(lab_events_sampled_sub['itemid'] == 51491) & (lab_events_sampled_sub['value'].isin([' '])), ['value', 'valuenum']] = 6.0

lab_events_sampled_sub.loc[(lab_events_sampled_sub['itemid'] == 51498) & (lab_events_sampled_sub['value'].isin(['>1.070'])), ['value', 'valuenum']] = 1.080

lab_events_sampled_sub.loc[(lab_events_sampled_sub['itemid'] == 51275) & (lab_events_sampled_sub['value'].isin(['ERROR'])), ['value', 'valuenum']] = np.nan

lab_events_sampled_sub.loc[(lab_events_sampled_sub['itemid'] == 51484) & (lab_events_sampled_sub['value'].astype(float) > 0), 'flag'] = 'abnormal'
lab_events_sampled_sub.loc[(lab_events_sampled_sub['itemid'] == 51478) & (lab_events_sampled_sub['value'].astype(float) > 0), 'flag'] = 'abnormal'

lab_events_sampled_sub.loc[(lab_events_sampled_sub['itemid'] == 51466) & (lab_events_sampled_sub['value'].astype(float) > 0), 'flag'] = 'abnormal'


In [20]:
# Check on differences between value and valuenum columns

lab_events_sampled_sub.loc[((lab_events_sampled_sub['value'].astype(str) + str('.0')) != lab_events_sampled_sub['valuenum'].astype(str)) 
                           & ((lab_events_sampled_sub['value'].astype(str) + str('0')) != lab_events_sampled_sub['valuenum'].astype(str)) 
                           & ((lab_events_sampled_sub['value'].astype(str)) != lab_events_sampled_sub['valuenum'].astype(str)) 
                           & ((lab_events_sampled_sub['value'].astype(str)) != lab_events_sampled_sub['valuenum'].astype(str) + str('0')) 
                           & ((str('0') + lab_events_sampled_sub['value'].astype(str)) != lab_events_sampled_sub['valuenum'].astype(str))
                           & ((lab_events_sampled_sub['value'].astype(str)) != round(lab_events_sampled_sub['valuenum'], 10).astype(str)), ]


,labevent_id,subject_id,hadm_id,specimen_id,itemid,charttime,storetime,value,valuenum,valueuom,ref_range_lower,ref_range_upper,flag,priority,comments
67219781,77231095,16195081,21878501.0,38616318,51498,2119-04-06 01:08:00,2119-04-06 01:14:00,1.000,1.0,,1.001,1.035,abnormal,STAT,NaN


## Code to look into specific test values for cleaning

In [165]:
# lab_events_sampled_sub[lab_events_sampled_sub['value'] == '>12*.  ']

In [166]:
# # Used to investigate bad data
# pd.set_option('display.max_rows', 200)

# bad_id = 51003

# # lab_events_sampled_sub[lab_events_sampled_sub['itemid'] == bad_id].value.value_counts().reset_index().sort_values(by = 'index')
# # lab_events_sampled_sub[lab_events_sampled_sub['itemid'] == bad_id][['value', 'comments']].drop_duplicates()
# # lab_events_sampled_sub[lab_events_sampled_sub['itemid'] == bad_id][['flag', 'comments', 'value', 'valuenum', 'ref_range_lower', 'ref_range_upper']].drop_duplicates()
# lab_events_sampled_sub.loc[(lab_events_sampled_sub['itemid'] == bad_id) & (lab_events_sampled_sub['value'].isnull()), ][['value', 'valuenum', 'comments']].drop_duplicates()


In [167]:
# pd.set_option('display.max_rows', 20)

## Get Aggregate Test Values

In [24]:
%time lab_events_sampled_sub_grouped = lab_events_sampled_sub.groupby(['subject_id', 'hadm_id', 'itemid']).agg({'itemid' : ['count'], 'valuenum' : ['min', 'max', 'mean'], 'flag' : ['count'], 'ref_range_lower' : ['min'], 'ref_range_upper' : ['min']})
# 'comments' : ['unique'], 


CPU times: user 223 ms, sys: 34.9 ms, total: 258 ms
Wall time: 257 ms


In [25]:
lab_events_sampled_sub_grouped.reset_index(inplace = True)
lab_events_sampled_sub_grouped.columns = ['subject_id', 'hadm_id', 'itemid', 'count', 'min', 'max', 'mean', 'abn_percent', 'range_min', 'range_max']
# 'comments', 


In [137]:
missing_vals = lab_events_sampled_sub_grouped.isnull().sum(axis = 0) 
missing_vals[missing_vals > 0].sort_values()

min     1267
max     1267
mean    1267
dtype: int64

In [141]:
lab_events_sampled_sub_grouped.loc[lab_events_sampled_sub_grouped['min'].isnull(), ].itemid.value_counts()

# lab_events_sampled_sub_grouped.loc[(lab_events_sampled_sub_grouped['min'].isnull()) & (lab_events_sampled_sub_grouped['itemid'] == 51003), ]#.subject_id.nunique()
# # lab_events_sampled_sub_grouped.subject_id.nunique()

# lab_events_sampled_sub_grouped.loc[lab_events_sampled_sub_grouped['subject_id'] == 10002264, ]


51003    989
50911    207
51498     33
50885     12
50861     10
51275      8
51274      6
51277      1
51265      1
Name: itemid, dtype: int64

In [27]:
lab_events_sampled_sub_grouped['below_min'] = np.where(lab_events_sampled_sub_grouped['min'] < lab_events_sampled_sub_grouped['range_min'], lab_events_sampled_sub_grouped['range_min'] - lab_events_sampled_sub_grouped['min'], 0)
lab_events_sampled_sub_grouped['above_max'] = np.where(lab_events_sampled_sub_grouped['max'] > lab_events_sampled_sub_grouped['range_max'], lab_events_sampled_sub_grouped['max'] - lab_events_sampled_sub_grouped['range_max'], 0)
lab_events_sampled_sub_grouped['abn_percent'] = lab_events_sampled_sub_grouped['abn_percent'] / lab_events_sampled_sub_grouped['count']


## Merge Admissions data so we can group by ethnicity/gender to grab average range min and max

In [29]:
# Here is where we merge Admissions

# lab_events_sampled_sub_grouped.merge(d_icd_diagnoses, left_index = True, left_on = "index", right_on = 'icd_code', how = 'left')


In [30]:
# For now aggregate all patients together

lab_range_min_max = lab_events_sampled_sub_grouped.groupby('itemid').agg({'range_min' : 'mean', 'range_max' : 'mean'})
lab_range_min_max.reset_index(inplace = True)

lab_range_dic = dict(zip(lab_range_min_max['itemid'], zip(lab_range_min_max['range_min'], lab_range_min_max['range_max'])))
lab_range_dic


{50861: (0.0, 40.0),
 50863: (37.366042345276874, 125.65757328990227),
 50868: (8.355709595038308, 19.572053994892375),
 50878: (0.0, 40.0),
 50882: (22.0, 32.0),
 50885: (0.0, 4.034812141795797),
 50893: (8.406765327696299, 10.290761099366144),
 50902: (96.0, 108.0),
 50911: (0.0, 10.0),
 50912: (0.44450771491552543, 1.1445077149154852),
 50931: (69.86519840516424, 100.84203531422062),
 50960: (1.598434891485942, 2.593739565943461),
 50970: (2.7223794212220396, 4.536227224008577),
 50971: (3.328334534967258, 5.142501802450993),
 50983: (133.35400797968808, 145.35400797968808),
 51003: (0.0, 0.009999999999999912),
 51006: (6.0, 20.0),
 51221: (37.33020850169711, 49.35930176175852),
 51222: (12.559139072847234, 16.73243377483429),
 51237: (0.9000000000000571, 1.0999999999999366),
 51248: (26.327974809413323, 31.912993039443155),
 51249: (31.322671528007955, 35.64534305601591),
 51250: (81.65197215777262, 98.34802784222738),
 51265: (150.0, 425.67536517314954),
 51274: (9.747572815534639

Pivot the table so we have feature columns related to test results

In [31]:
lab_events_sampled_sub_grouped['new_index'] = list(zip(lab_events_sampled_sub_grouped['subject_id'], lab_events_sampled_sub_grouped['hadm_id']))


In [32]:
%time lab_events_sampled_pivot = lab_events_sampled_sub_grouped.pivot(index = 'new_index', columns = 'itemid', values = ['min', 'max', 'mean', 'abn_percent', 'below_min', 'above_max'])
# 'comments', 


CPU times: user 61.7 ms, sys: 9.91 ms, total: 71.6 ms
Wall time: 70.6 ms


In [33]:
lab_events_sampled_pivot = lab_events_sampled_pivot.reset_index()
lab_events_sampled_pivot['subject_id'], lab_events_sampled_pivot['hadm_id'] = zip(*lab_events_sampled_pivot['new_index'])
lab_events_sampled_pivot.drop(['new_index'], axis = 1, inplace = True)


/opt/anaconda3/lib/python3.7/site-packages/pandas/core/generic.py:3936: PerformanceWarning: dropping on a non-lexsorted multi-index without a level parameter may impact performance.
  obj = obj._drop_axis(labels, axis, level=level, errors=errors)


In [34]:
missing_vals = lab_events_sampled_pivot.isnull().sum(axis = 0) 
missing_vals[missing_vals > 0].sort_values()

             itemid
mean         51221     1980
min          51221     1980
max          51221     1980
above_max    51221     1980
abn_percent  51221     1980
                       ... 
min          50911     6912
mean         50911     6912
             51003     7460
max          51003     7460
min          51003     7460
Length: 240, dtype: int64

## Start Data Imputation

Copy table, retain original for Random Forest

In [35]:
lab_events_impute = lab_events_sampled_pivot.copy()

In [36]:
d_labitems[d_labitems['itemid'].isin(itemid_sub_sample)]

,itemid,label,fluid,category,loinc_code
52,50861,Alanine Aminotransferase (ALT),Blood,Chemistry,1742-6
76,50863,Alkaline Phosphatase,Blood,Chemistry,6768-6
115,50868,Anion Gap,Blood,Chemistry,1863-0
153,50878,Asparate Aminotransferase (AST),Blood,Chemistry,1920-8
212,50882,Bicarbonate,Blood,Chemistry,1963-8
222,51464,Bilirubin,Urine,Hematology,5770-3
229,50885,"Bilirubin, Total",Blood,Chemistry,1975-2
258,51466,Blood,Urine,Hematology,NaN
282,50893,"Calcium, Total",Blood,Chemistry,2000-8
442,50902,Chloride,Blood,Chemistry,2075-0


In [37]:
%%time

np.random.seed(0)

for labitem in lab_range_dic:
    for ind in lab_events_impute[lab_events_impute[('mean', labitem)].isnull()].index:
        val_max = lab_range_dic[labitem][1]
        val_min = lab_range_dic[labitem][0]

        val_ave = (val_max + val_min) / 2
        val_std = (val_max - val_ave) * .333

        ran_vals = np.random.normal(val_ave, val_std, 50)
        impute_min = min(ran_vals)
        impute_max = max(ran_vals)
        impute_mean = np.mean(ran_vals)
        
        val_min = lab_events_impute[('min', labitem)][ind]
        val_max = lab_events_impute[('max', labitem)][ind]
        val_mean = lab_events_impute[('mean', labitem)][ind]
        
        lab_events_impute[('min', labitem)][ind] = np.where(np.isnan(val_min), impute_min, val_min)
        lab_events_impute[('max', labitem)][ind] = np.where(np.isnan(val_max), impute_max, val_max)
        lab_events_impute[('mean', labitem)][ind] = np.where(np.isnan(val_mean), impute_mean, val_mean)
#         patient[('mean', labitem)].replace(np.nan, impute_mean)
#         patient[('abn_count', labitem)].replace(np.nan, 0)
# 5k patients
# 10 items - 2.5s
# 20 items - 6s
# 30 items - 28.6s

# 10k patients
# 40 items - 1m 34s

/opt/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:20: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
/opt/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:21: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
/opt/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:22: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy


CPU times: user 1min 30s, sys: 329 ms, total: 1min 30s
Wall time: 1min 30s


In [38]:
for lab_itemid in lab_range_dic:
    lab_events_impute[( 'above_max', lab_itemid)] = lab_events_impute[( 'above_max', lab_itemid)].replace(np.nan, 0)
    lab_events_impute[( 'below_min', lab_itemid)] = lab_events_impute[( 'below_min', lab_itemid)].replace(np.nan, 0)
    lab_events_impute[( 'abn_percent', lab_itemid)] = lab_events_impute[( 'abn_percent', lab_itemid)].replace(np.nan, 0)
#     lab_events_sampled_pivot[( 'comments', lab_itemid)] = lab_events_sampled_pivot[( 'comments', lab_itemid)].replace(np.nan, np.array(np.nan))


In [39]:
missing_vals = lab_events_impute.isnull().sum(axis = 0) 
missing_vals[missing_vals > 0].sort_values()

Series([], dtype: int64)

In [40]:
lab_events_impute

min                                                         \
itemid      50861       50863      50868      50878      50882     50885   
0        2.997088   53.363795  10.369141  -2.706156  23.468429  4.100000   
1       36.000000   75.000000  15.000000  32.000000  14.000000  0.400000   
2        8.502958   51.899458   9.000000   6.138574  23.000000  0.745549   
3       10.068224   48.020546   9.310767   3.595374  23.373409  2.600000   
4        5.192135   42.016499   9.261099   6.635144  24.476791  7.600000   
...           ...         ...        ...        ...        ...       ...   
8162     5.450012   60.737503   8.333361   6.911677  23.766951  1.047266   
8163     9.210654   42.542491  12.000000  -6.983840  26.000000  0.506439   
8164     9.000000   64.000000  10.000000  11.000000  16.000000  0.300000   
8165    15.000000   57.000000   5.000000  25.000000  22.000000  0.800000   
8166    28.000000  112.000000   7.000000  51.000000  27.000000  1.000000   

                                                   ... above_max              \
itemid     50893       50902      50911     50912  ...     51484 51486 51487   
0       8.659014   96.556417   1.614418  0.510737  ...       0.0   0.0   0.0   
1       7.700000   95.000000  44.000000  1.100000  ...       0.0   4.0   0.0   
2       8.791798  103.000000   1.358330  1.000000  ...       0.0   0.0   0.0   
3       8.511920   96.704762   2.099609  0.425821  ...       0.0   0.0   0.0   
4       8.587909   97.204544   0.227240  0.536863  ...       0.0   0.0   0.0   
...          ...         ...        ...       ...  ...       ...   ...   ...   
8162    8.523331   97.682902   1.729202  0.593280  ...       0.0   0.0   0.0   
8163    9.100000  104.000000   0.650965  1.500000  ...       0.0   0.0   0.0   
8164    7.500000   86.000000   2.382568  0.800000  ...       0.0   0.0   0.0   
8165    7.800000   92.000000   0.772004  0.500000  ...       0.0   4.0   0.0   
8166    8.100000   90.000000   0.938207  0.800000  ...       0.0   0.0   0.0   

                                     subject_id     hadm_id  
itemid 51491 51492 51498 51506 51514                         
0        0.0   0.0   0.0   0.0   0.0   10002264  23205369.0  
1        0.0  30.0   0.0   1.0   0.0   10002495  24982426.0  
2        0.0   0.0   0.0   0.0   0.0   10002527  29112696.0  
3        0.0   0.0   0.0   0.0   0.0   10005078  24046197.0  
4        0.0   0.0   0.0   0.0   0.0   10006062  21916409.0  
...      ...   ...   ...   ...   ...        ...         ...  
8162     0.0   0.0   0.0   0.0   0.0   19991798  26825654.0  
8163     0.0   0.0   0.0   0.0   0.0   19995478  24108472.0  
8164     0.0   0.0   0.0   0.0   0.0   19996783  25894657.0  
8165     0.0  30.0   0.0   1.0   1.0   19997886  20793010.0  
8166     0.0   0.0   0.0   0.0   1.0   19999204  29046609.0  

[8167 rows x 242 columns]

In [41]:
lab_events_impute_KNN = lab_events_sampled_pivot.copy()

In [42]:
%%time

from sklearn.impute import KNNImputer
imputer = KNNImputer(n_neighbors=math.trunc((lab_events_impute_KNN.shape[0] ** 0.5)))
lab_events_impute_KNN_imputed = imputer.fit_transform(lab_events_impute_KNN)

# 3min 8s

CPU times: user 2min 22s, sys: 23.8 s, total: 2min 46s
Wall time: 2min 42s


In [43]:
lab_events_impute_KNN_imputed = pd.DataFrame(lab_events_impute_KNN_imputed)
lab_events_impute_KNN_imputed.columns = lab_events_impute_KNN.columns
lab_events_impute_KNN_imputed

min                                                         \
itemid      50861       50863      50868      50878      50882     50885   
0       77.900000   94.677778  11.900000  53.022222  21.688889  4.100000   
1       36.000000   75.000000  15.000000  32.000000  14.000000  0.400000   
2       56.800000   99.766667   9.000000  73.977778  23.000000  1.601111   
3       91.522222  104.700000  11.655556  61.888889  22.288889  2.600000   
4       51.144444   91.933333  11.988889  36.933333  22.788889  7.600000   
...           ...         ...        ...        ...        ...       ...   
8162    70.655556  101.344444  11.522222  69.533333  23.200000  2.161111   
8163    37.955556  103.400000  12.000000  37.544444  26.000000  1.578889   
8164     9.000000   64.000000  10.000000  11.000000  16.000000  0.300000   
8165    15.000000   57.000000   5.000000  25.000000  22.000000  0.800000   
8166    28.000000  112.000000   7.000000  51.000000  27.000000  1.000000   

                                                   ... above_max            \
itemid     50893       50902      50911     50912  ...     51484     51486   
0       8.208889  100.677778   7.866667  0.852222  ...  8.611111  1.044444   
1       7.700000   95.000000  44.000000  1.100000  ...  0.000000  4.000000   
2       8.488889  103.000000   5.044444  1.000000  ...  6.666667  1.277778   
3       8.306667   99.522222   8.188889  0.832222  ...  8.166667  0.933333   
4       8.206667  100.800000   7.877778  0.877778  ...  7.888889  0.755556   
...          ...         ...        ...       ...  ...       ...       ...   
8162    8.250000   99.744444   7.244444  0.895556  ...  7.611111  0.988889   
8163    9.100000  104.000000   6.200000  1.500000  ...  5.666667  1.111111   
8164    7.500000   86.000000   7.333333  0.800000  ...  0.000000  0.000000   
8165    7.800000   92.000000   5.433333  0.500000  ...  0.000000  4.000000   
8166    8.100000   90.000000   5.244444  0.800000  ...  0.000000  0.000000   

                                                                     \
itemid     51487     51491      51492     51498     51506     51514   
0       0.055556  0.005556  38.222222  0.000678  0.411111  0.388889   
1       0.000000  0.000000  30.000000  0.000000  1.000000  0.000000   
2       0.066667  0.016667  44.777778  0.000822  0.488889  0.722222   
3       0.044444  0.005556  32.111111  0.000689  0.344444  0.322222   
4       0.022222  0.005556  39.611111  0.000889  0.388889  0.633333   
...          ...       ...        ...       ...       ...       ...   
8162    0.044444  0.005556  37.222222  0.000844  0.433333  0.866667   
8163    0.055556  0.000000  34.111111  0.000089  0.516667  0.544444   
8164    0.000000  0.000000   0.000000  0.000000  0.000000  0.000000   
8165    0.000000  0.000000  30.000000  0.000000  1.000000  1.000000   
8166    0.000000  0.000000   0.000000  0.000000  0.000000  1.000000   

        subject_id     hadm_id  
itemid                          
0       10002264.0  23205369.0  
1       10002495.0  24982426.0  
2       10002527.0  29112696.0  
3       10005078.0  24046197.0  
4       10006062.0  21916409.0  
...            ...         ...  
8162    19991798.0  26825654.0  
8163    19995478.0  24108472.0  
8164    19996783.0  25894657.0  
8165    19997886.0  20793010.0  
8166    19999204.0  29046609.0  

[8167 rows x 242 columns]

In [ ]:
for lab_itemid in lab_range_dic:
    lab_events_impute_KNN_imputed[( 'above_max', lab_itemid)] = lab_events_impute_KNN_imputed[( 'above_max', lab_itemid)].replace(np.nan, 0)
    lab_events_impute_KNN_imputed[( 'below_min', lab_itemid)] = lab_events_impute_KNN_imputed[( 'below_min', lab_itemid)].replace(np.nan, 0)
    lab_events_impute_KNN_imputed[( 'abn_percent', lab_itemid)] = lab_events_impute_KNN_imputed[( 'abn_percent', lab_itemid)].replace(np.nan, 0)
#     lab_events_sampled_pivot[( 'comments', lab_itemid)] = lab_events_sampled_pivot[( 'comments', lab_itemid)].replace(np.nan, np.array(np.nan))



## Standardize the Data for use in Models

In [44]:
from sklearn.preprocessing import StandardScaler

scaler = StandardScaler()

In [45]:
labs_scaled = lab_events_impute
labs_scaled_KNN = lab_events_impute_KNN_imputed
labs_scaled_missing_data = lab_events_sampled_pivot

In [60]:
scale_cols = ~labs_scaled.columns.isin([('subject_id', ''), ('hadm_id', '')])

scaler.fit(labs_scaled.loc[:, scale_cols])
labs_scaled.loc[:, scale_cols] = scaler.transform(labs_scaled.loc[:, scale_cols])

scaler.fit(labs_scaled_KNN.loc[:, scale_cols])
labs_scaled_KNN.loc[:, scale_cols] = scaler.transform(labs_scaled_KNN.loc[:, scale_cols])

scaler.fit(labs_scaled_missing_data.loc[:, scale_cols])
labs_scaled_missing_data.loc[:, scale_cols] = scaler.transform(labs_scaled_missing_data.loc[:, scale_cols])


## Save Lab Results for Models

In [62]:
# Save non-imputed file as well for forest models

pos_final_test = labs_scaled.merge(admissions_sample, how='left', left_on='hadm_id', right_on='hadm_id')
pos_KNN_final_test = labs_scaled_KNN.merge(admissions_sample, how='left', left_on='hadm_id', right_on='hadm_id')
pos_final_missing_data_test = labs_scaled_missing_data.merge(admissions_sample, how='left', left_on='hadm_id', right_on='hadm_id')


/opt/anaconda3/lib/python3.7/site-packages/pandas/core/reshape/merge.py:618: UserWarning: merging between different levels can give an unintended result (2 levels on the left, 1 on the right)
  warnings.warn(msg, UserWarning)


In [63]:
pos_final_test.to_csv('pos_final_test.csv.gz', compression="gzip")
pos_KNN_final_test.to_csv('pos_KNN_final_test.csv.gz', compression="gzip")
pos_final_missing_data_test.to_csv('pos_final_missing_data_test.csv.gz', compression="gzip")


## Check for Multicollinearity

In [64]:
from statsmodels.stats.outliers_influence import variance_inflation_factor
from statsmodels.tools.tools import add_constant

In [65]:
%%time

labs_scaled_features = labs_scaled.loc[:, ('min', 50868) : ('above_max', 51484)]
add_constant(labs_scaled_features)

high_vif = pd.Series([variance_inflation_factor(labs_scaled_features.values, i) 
               for i in range(labs_scaled_features.shape[1])], 
              index=labs_scaled_features.columns).sort_values(ascending = False)

high_vif.head(20)


/opt/anaconda3/lib/python3.7/site-packages/statsmodels/stats/outliers_influence.py:193: RuntimeWarning: divide by zero encountered in double_scalars
  vif = 1. / (1. - r_squared_i)
/opt/anaconda3/lib/python3.7/site-packages/statsmodels/regression/linear_model.py:1687: RuntimeWarning: invalid value encountered in double_scalars
  return 1 - self.ssr/self.uncentered_tss


CPU times: user 4min 1s, sys: 27.1 s, total: 4min 28s
Wall time: 45.6 s


           itemid
above_max  51484              inf
max        51464              inf
above_max  51478              inf
max        51484              inf
           51478              inf
           51466              inf
above_max  51464              inf
           51466              inf
max        51003     7.760296e+05
above_max  51003     7.757782e+05
max        50878     1.155922e+04
above_max  50878     1.150424e+04
max        50861     4.008560e+03
above_max  50861     3.980083e+03
max        51237     1.036024e+03
above_max  51237     9.592607e+02
max        51274     5.082725e+02
mean       51248     5.018567e+02
max        50931     4.900186e+02
above_max  50931     4.738349e+02
dtype: float64

In [171]:
labs_scaled_features_limited = labs_scaled_features.copy()
labs_scaled_features_limited.drop([('above_max', 51484),
                                  ('above_max', 51478),
                                  ('above_max', 51466),
                                  ('above_max', 51464),
                                  ('max', 51003),
                                  ('max', 50878),
                                  ('max', 50861),
                                  ('max', 51237),
                                  ('mean', 51248),
                                  ('max', 50931),
                                  ('mean', 51222),
                                  ('max', 51274),
                                  ('mean', 51279),
                                  ('mean', 51274),
                                  ('mean', 51478),
                                  ('mean', 50885),
                                  ('mean', 51250),
                                  ('max', 50911),
                                  ('mean', 51003),
                                  ('mean', 51466),
                                  ('mean', 51484),
                                  ('mean', 51486),
                                  ('mean', 51277),
                                  ('mean', 51221),
                                  ('max', 50912),
                                  ('max', 51275),
                                  ('mean', 51492),
                                  ('mean', 51514),
                                  ('mean', 51006)], axis = 1, inplace = True)


In [172]:
%%time

high_vif = pd.Series([variance_inflation_factor(labs_scaled_features_limited.values, i) 
               for i in range(labs_scaled_features_limited.shape[1])], 
              index=labs_scaled_features_limited.columns).sort_values(ascending = False)

high_vif.head(20)


CPU times: user 3min 9s, sys: 31.6 s, total: 3min 40s
Wall time: 38.7 s


             itemid
min          51222     82.262478
mean         51506     80.965804
             51464     77.577210
             50911     71.498252
             51301     69.382835
             51249     67.395665
min          51279     65.890825
max          51222     64.614807
mean         50912     62.892656
max          51279     57.302073
min          51248     55.991435
below_min    51222     55.654000
max          51248     55.000721
below_min    51221     53.702817
mean         51487     52.928935
below_min    51279     52.823280
mean         51265     50.400899
max          51301     45.575576
min          51250     44.780498
abn_percent  51274     44.297510
dtype: float64